# Lab | Random Forests
For this lab, you will be using the CSV files provided in the files_for_lab folder.

Instructions
Apply the Random Forests algorithm but this time only by upscaling the data.
Discuss the output and its impact in the bussiness scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the bussiness?

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import load_boston
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [2]:
numerical = pd.read_csv('files_for_lab/numerical.csv')
categorical = pd.read_csv('files_for_lab/categorical.csv')
targets = pd.read_csv('files_for_lab/target.csv')
data = pd.concat([numerical, categorical, targets], axis = 1)

In [3]:
data.isna().sum().sum()

0

In [4]:
data.TARGET_B.value_counts(dropna=False)

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [5]:
y = data['TARGET_B']
X = data.drop(['TARGET_B'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(object)

# Note: we need to do train/test split before downsampling, and then only downsample the training set - Why?
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [6]:
y_train = y_train.reset_index(drop=True) 
y_test = y_test.reset_index(drop=True) 

In [7]:
# X train/ X test - numericals and categoricals transformation

numericalX_train = X_train.select_dtypes(np.number)
numericalX_test = X_test.select_dtypes(np.number)
categoricalX_train = X_train.select_dtypes(object)
categoricalX_test = X_test.select_dtypes(object)

#we OneHotEncode the categoricals and fit on X train
encoder = OneHotEncoder(drop='first').fit(categoricalX_train)
encoded_cat_X_train = encoder.transform(categoricalX_train).toarray()
encoded_cat_X_train = pd.DataFrame(encoded_cat_X_train)
#OneHotEncoding X test
encoded_cat_X_test = encoder.transform(categoricalX_test).toarray()
encoded_cat_X_test = pd.DataFrame(encoded_cat_X_test)

# add scaling step for numericals
scaler = StandardScaler()
numericalX_train_scaled = scaler.fit_transform(numericalX_train)
numericalX_train_scaled = pd.DataFrame(numericalX_train_scaled, columns=numericalX.columns)
numericalX_test_scaled = scaler.fit_transform(numericalX_test)
numericalX_test_scaled = pd.DataFrame(numericalX_test_scaled,columns=numericalX.columns)

In [8]:
X_train = pd.concat([numericalX_train_scaled, encoded_cat_X_train], axis = 1)
X_test = pd.concat([numericalX_test_scaled, encoded_cat_X_test], axis = 1)

In [9]:
# for upsampling we need to temporarily concat X_train and y_train
trainset = pd.concat([X_train, y_train], axis=1)

In [10]:
# quicker way to upsample category 1:
category_1 = trainset[trainset['TARGET_B']==1].sample(len(trainset[trainset['TARGET_B']==0]), replace = True)
print(category_1.shape)

category_0 = trainset[trainset['TARGET_B']== 0 ]
trainset_new = pd.concat([category_0, category_1], axis = 0)
trainset_new = trainset_new.sample(frac =1) #randomize the rows
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']
#data = data.reset_index(drop=True)
print(X_train.shape)

(72486, 356)
(144972, 355)


In [11]:
y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

# Now we can remove the column target d from the set of features
X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

In [12]:
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20,
                             max_samples=0.2,
                             random_state = 42) # you can also try to optimize the bootstraping parameters
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.6211268382860139
0.5822983807577425


0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[10546,  7537],
       [  434,   566]], dtype=int64)

In [13]:
7544/(7544+571)

0.9296364756623536

In [14]:
429/(10539+429)

0.03911378555798687

In [15]:
#In this case, the cost of a false positive (those people who did donate, but will not get an ad)
#is bigger than that of a false negative (people who didnt donate, but got a mail). 

LR = LogisticRegression(max_iter=1000)
LR.fit(X_train, y_train)
pred = LR.predict(X_test)
print(LR.score(X_test, y_test))

print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

In [17]:
email =0.68
mean_don = data['TARGET_D'][data['TARGET_D'] !=0].mean()
false_positive = round(((7544*mean_don)-email*7544), 2)
false_negative = round(429*email,2)
print('The cost of a false positive is', false_positive, ', the difference between what would have been spent with emails and what\n could have been donated, should donors had gotten emails.')
print('The cost of a false negative is', false_negative, ', the amount spent with emails to no-donors.')

The cost of a false positive is 112740.13 , the difference between what would have been spent with emails and what
 could have been donated, should donors had gotten emails.
The cost of a false negative is 291.72 , the amount spent with emails to no-donors.


### Trying Kbest to select numerical features

In [18]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [19]:
#because Standard scaling gives negative numbers and kbest doesnt work with negative input
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(numerical)

In [20]:
X = numerical_scaled
y = targets['TARGET_B']

In [21]:
kbest = SelectKBest(chi2, k=10).fit_transform(X, y)
# Here we choose 10 so that is easier to analyze results later, as we will see
selected = pd.DataFrame(kbest)
selected.head()

# To check the scores
model = SelectKBest(chi2, k=10).fit(X, y)
df = pd.DataFrame(data = model.scores_, columns = ['score'])
df['Column'] = numerical.columns
print(df.sort_values(by = ['score'], ascending = False).head(10))

#A bigger score suggest a better usefulness for that column


cols = df.sort_values(by = ['score'], ascending = False).head(10)['Column']
cols

selected.columns = cols
selected.head()

          score    Column
313  210.290810    RFA_2F
305   27.516508  CARDGIFT
110   26.474583      HVP1
111   25.757492      HVP2
112   21.545080      HVP3
115   19.261394      HVP6
113   14.874056      HVP4
22    13.034181      ETH2
129   12.581538       RP1
130   11.022637       RP2


Column,RFA_2F,CARDGIFT,HVP1,HVP2,HVP3,HVP6,HVP4,ETH2,RP1,RP2
0,0.010101,0.020202,0.070707,0.131313,0.272727,0.000000,0.020202,0.050505,0.341463,1.000000
1,0.000000,0.979798,0.989899,0.989899,0.989899,0.949495,0.919192,0.919192,0.024390,0.333333
2,0.020202,0.000000,0.010101,0.060606,0.181818,0.000000,0.000000,0.020202,0.341463,1.000000
3,0.000000,0.101010,0.252525,0.505051,0.696970,0.101010,0.010101,0.080808,0.170732,1.000000
4,0.989899,0.000000,0.010101,0.020202,0.161616,0.000000,0.252525,0.585859,0.195122,0.333333


In [22]:
num_columns= selected.columns

In [23]:
numerical2 = numerical[num_columns] #rerun code with new numerical

In [24]:
data2 = pd.concat([numerical2, categorical, targets], axis = 1)

In [25]:
y = data2['TARGET_B']
X = data2.drop(['TARGET_B'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(object)

# Note: we need to do train/test split before downsampling, and then only downsample the training set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [26]:
y_train = y_train.reset_index(drop=True) 
y_test = y_test.reset_index(drop=True) 

In [27]:
# X train/ X test - numericals and categoricals transformation

numericalX_train = X_train.select_dtypes(np.number)
numericalX_test = X_test.select_dtypes(np.number)
categoricalX_train = X_train.select_dtypes(object)
categoricalX_test = X_test.select_dtypes(object)

#we OneHotEncode the categoricals and fit on X train
encoder = OneHotEncoder(drop='first').fit(categoricalX_train)
encoded_cat_X_train = encoder.transform(categoricalX_train).toarray()
encoded_cat_X_train = pd.DataFrame(encoded_cat_X_train)
#OneHotEncoding X test
encoded_cat_X_test = encoder.transform(categoricalX_test).toarray()
encoded_cat_X_test = pd.DataFrame(encoded_cat_X_test)

# add scaling step for numericals
scaler = StandardScaler()
numericalX_train_scaled = scaler.fit_transform(numericalX_train)
numericalX_train_scaled = pd.DataFrame(numericalX_train_scaled, columns=numericalX.columns)
numericalX_test_scaled = scaler.transform(numericalX_test)
numericalX_test_scaled = pd.DataFrame(numericalX_test_scaled,columns=numericalX.columns)

In [28]:
X_train = pd.concat([numericalX_train_scaled, encoded_cat_X_train], axis = 1)
X_test = pd.concat([numericalX_test_scaled, encoded_cat_X_test], axis = 1)

In [29]:
# for upsampling we need to temporarily concat X_train and y_train
trainset = pd.concat([X_train, y_train], axis=1)

In [30]:
# quicker way to upsample category 1:
category_1 = trainset[trainset['TARGET_B']==1].sample(len(trainset[trainset['TARGET_B']==0]), replace = True)
print(category_1.shape)

category_0 = trainset[trainset['TARGET_B']== 0 ]
trainset_new = pd.concat([category_0, category_1], axis = 0)
trainset_new = trainset_new.sample(frac =1) #randomize the rows
X_train = trainset_new.drop(['TARGET_B'], axis=1)
y_train = trainset_new['TARGET_B']
#data = data.reset_index(drop=True)
print(X_train.shape)

(72486, 51)
(144972, 50)


In [31]:
y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

# Now we can remove the column target d from the set of features
X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

In [32]:
#Romeo run the Grid Search and found out the best ones, however, the score was too close to 1, so it is not that reliable. 
clf = RandomForestClassifier(max_depth=None,
                             criterion = 'entropy',
                             min_samples_split=2,
                             min_samples_leaf =2,
                             max_samples=0.3,
                             random_state = 42) # you can also try to optimize the bootstraping parameters
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0.9999586126976243
0.9468112980139392


0    18083
1     1000
Name: TARGET_B, dtype: int64

array([[18064,    19],
       [  996,     4]], dtype=int64)

LR = LogisticRegression(max_iter=1000)
LR.fit(X_train, y_train)
pred = LR.predict(X_test)
print(LR.score(X_test, y_test))

print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

# Lab | Final regression model in "Health Care for All" Case
Instructions
At this point, we have created a model to predict who will make a donation and who won't. But, what about the ammount of money that each person will give? In this lab, subset those that made a donation and use that subset to create a model to predict how much money will they give.

Evaluate the result of your model and estimate how much better the result are for the bussiness in comparison with the naive scenario we discuss on Monday.

In [33]:
data3 = data[~data['TARGET_D'].isin([0])]

In [34]:
data3 = data3.drop('TARGET_B', axis = 1)

In [35]:
data3.shape

(4843, 338)

In [36]:
y = data3['TARGET_D']
X = data3.drop(['TARGET_D'], axis = 1)

cat = X.select_dtypes(object)
num = X.select_dtypes(np.number)

scaler = StandardScaler()
num_scaled = scaler.fit_transform(num)
num_scaled = pd.DataFrame(num_scaled,columns=num.columns)

encoder = OneHotEncoder(drop='first').fit(cat)
encoded_cat = encoder.transform(cat).toarray()
encoded_cat = pd.DataFrame(encoded_cat)

X = pd.concat([num_scaled, encoded_cat], axis = 1)

In [37]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)

In [38]:
# X train transform
X_train_num = X_train.select_dtypes(np.number)
X_train_cat = X_train.select_dtypes(object)

encoder = OneHotEncoder(drop='first').fit(X_train_cat)
encoded_X_train_cat = encoder.transform(X_train_cat).toarray()
encoded_X_train_cat = pd.DataFrame(encoded_X_train_cat)

scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train_num)
X_train_num_scaled = pd.DataFrame(X_train_num_scaled)

In [40]:
#X val transform
X_val_num = X_val.select_dtypes(np.number)
X_val_cat = X_val.select_dtypes(object)

encoder = OneHotEncoder(drop='first').fit(X_val_cat)
encoded_X_val_cat = encoder.transform(X_val_cat).toarray()
encoded_X_val_cat = pd.DataFrame(encoded_X_val_cat)

X_val_num_scaled = scaler.transform(X_val_num)
X_val_num_scaled = pd.DataFrame(X_val_num_scaled)

In [41]:
X_train = pd.concat([X_train_num_scaled, encoded_X_train_cat], axis = 1)
X_val = pd.concat([X_val_num_scaled, encoded_X_val_cat], axis = 1)

In [42]:
X_val_cat.shape

(1211, 7)

In [64]:
model1 = DecisionTreeRegressor(max_depth=3,
                             criterion = 'mse',
                             min_samples_split=3,
                             min_samples_leaf =3,
                             random_state = 42)
model2 = LinearRegression()
model3 = KNeighborsRegressor(n_neighbors=3)

In [65]:
model_pipeline = [model1, model2, model3]
model_names = ['Decision Tree Regressor', 'Linear Regression', 'KNN']
scores = {}
for model, model_name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=3))
    scores[model_name] = mean_score
print(scores)

{'Decision Tree Regressor': 0.3638929687460279, 'Linear Regression': 0.4219748233557188, 'KNN': 0.01858626224302229}


In [66]:
list(zip(model_pipeline, model_names))

[(DecisionTreeRegressor(criterion='mse', max_depth=3, min_samples_leaf=3,
                        min_samples_split=3, random_state=42),
  'Decision Tree Regressor'),
 (LinearRegression(), 'Linear Regression'),
 (KNeighborsRegressor(n_neighbors=3), 'KNN')]

In [67]:
val_scores = {}

for model, model_name in zip(model_pipeline,model_names):
    model.fit(X_train, y_train)
    val_scores[model_name] = model.score(X_val,y_val)
print(val_scores)

{'Decision Tree Regressor': 0.45889046913400733, 'Linear Regression': -0.0691740484932255, 'KNN': -0.022315962701981107}
